In [1]:
import pandas as pd
import numpy as np

Всем привет! Поговорим про **groupby()**

_groupby()_ используется для группировки данных по одному или нескольким столбцам и подсчету по ним агрегирующих значений


Чаще всего используемые агрегированные функции: 

* **mean** - среднее
* **sum** - сумма
* **max** - максимум
* **min** - минимум
* **median** - медиана
* **count** - кол-во непустых значений
* **nunique** - кол-во уникальных значений (нужно проверить учитывается ли nan)  

Создадим датафрейм и будем смотреть, что с ним происходит :)

In [2]:
df_lst = {'Names': ['Sasha', 'Ann', 'Mikhail', 'Valera', 'Max', 'Ann', 'Nina', 'Pasha', 'Sasha', 'Mark'], 
          'Subj': ['math', 'math', 'rus', 'eng', 'rus', 'math', 'rus', 'eng', 'eng', 'math'],
          'Mark_1': [7, 6, 10, 6, 2, 3, 5, 7, 8, np.nan],
          'Mark_2': [5, 10, 2, 6, 7, 4, 9, 10, 9, np.nan]}
df = pd.DataFrame(data = df_lst)
df

,Names,Subj,Mark_1,Mark_2
0,Sasha,math,7.0,5.0
1,Ann,math,6.0,10.0
2,Mikhail,rus,10.0,2.0
3,Valera,eng,6.0,6.0
4,Max,rus,2.0,7.0
5,Ann,math,3.0,4.0
6,Nina,rus,5.0,9.0
7,Pasha,eng,7.0,10.0
8,Sasha,eng,8.0,9.0
9,Mark,math,NaN,NaN


P.S. все метрики вымыщленные и не имеют особого смысла :) 

# Группировка по одному полю
df.groupby('column_for_agg').function()
где 
- column_for_agg - столбец, по которому будем агрегировать 
- function - агрегирующая функция

В данном случае будет выведен полностью датафрейм, однако столбец _column_for_agg_ станет индексом датафрейма и агрегирующая функция _function_ будет применяться к каждому столбцу

Для каждого имени выведем максимум в каждом столбце:

In [3]:
df.groupby('Names').max() 

,Subj,Mark_1,Mark_2
Names,,,
Ann,math,6.0,10.0
Mark,math,NaN,NaN
Max,rus,2.0,7.0
Mikhail,rus,10.0,2.0
Nina,rus,5.0,9.0
Pasha,eng,7.0,10.0
Sasha,math,8.0,9.0
Valera,eng,6.0,6.0


Еще раз напомню, что при использовании **df.groupby('Names').max()** - максимум будет применяться к каждому столбцу

Если мы не будем использовать агрегирующую функцию и напишем просто df.groupby('Names') - это будет специальный объект, на который просто так не посмотришь

In [4]:
df.groupby('Names')

Как с ним работать будет в спринте "Исследовательский анализ данных"

# Группировка по одному полю с агрегирующей функцией, примененной к столбцу

df.groupby('column_for_agg')['column_for_create_cal'].function()

где
   * _column_for_agg_ - столбец, по которому будем агрегировать 
   * _column_for_create_cal_ - столбец, по которому будем вычислять значение
   * _function_ - агрегирующая функция

Для каждого студента посчитаем максимульную оценку **Mark_1** независимо от предмета

In [5]:
df.groupby('Names')['Mark_1'].max()

Names
Ann         6.0
Mark        NaN
Max         2.0
Mikhail    10.0
Nina        5.0
Pasha       7.0
Sasha       8.0
Valera      6.0
Name: Mark_1, dtype: float64

В данном случае мы группируем по одному полю, поэтому не принципиальны _[] скобки_. Но если их поставить, то ошибки не будет 

In [6]:
df.groupby(['Names'])['Mark_1'].max()

Names
Ann         6.0
Mark        NaN
Max         2.0
Mikhail    10.0
Nina        5.0
Pasha       7.0
Sasha       8.0
Valera      6.0
Name: Mark_1, dtype: float64

Выводит тоже самое :) 

Существует метод **agg**, с помощью которого так же можно посчитать значения:

In [7]:
df.groupby(['Names']).agg({'Mark_1' : 'max'})

,Mark_1
Names,
Ann,6.0
Mark,NaN
Max,2.0
Mikhail,10.0
Nina,5.0
Pasha,7.0
Sasha,8.0
Valera,6.0


Не забываем, что в данном случае **столбец**, по которому будем вычислять значения, **не берется в квадратные скобки**. Можно использовать агрегирующие функции в кавычках или вызывая их через numpy:

In [8]:
df.groupby(['Mark_1']).agg({'Names' : 'unique'})

,Names
Mark_1,
2.0,[Max]
3.0,[Ann]
5.0,[Nina]
6.0,"[Ann, Valera]"
7.0,"[Sasha, Pasha]"
8.0,[Sasha]
10.0,[Mikhail]


In [9]:
df.groupby(['Mark_1']).agg({'Names' : np.unique})

,Names
Mark_1,
2.0,Max
3.0,Ann
5.0,Nina
6.0,"[Ann, Valera]"
7.0,"[Pasha, Sasha]"
8.0,Sasha
10.0,Mikhail


Однако **agg** чаще используют, когда нужно посчитать агрегацию **по нескольким значениям**. Рассмотрим это дальше 

# Группировка по нескольким полям

df.groupby(['column_for_agg_1', 'column_for_agg_2'])['column_for_create_cal'].function()

где
   * _column_for_agg_1_ - столбец, по которому будем агрегировать 
   * _column_for_agg_2_ - столбец, по которому будем агрегировать 
   * _column_for_create_cal_ - столбец, по которому будем вычислять значение
   * _function_ - агрегирующая функция

In [10]:
df.groupby('Names', 'Subj')['Mark_1'].mean()

ValueError: No axis named Subj for object type <class 'pandas.core.frame.DataFrame'>

**!!!ОШИБКА!!!** - происходит потому, что мы пытаемся сделать группировку по нескольким полям, но не взяли их в квадратные скобки

**!!!ВЫВОД!!!** - не забываем про квадратные скобочки :) 

In [11]:
df.groupby(['Names', 'Subj'])['Mark_1'].mean()

Names    Subj
Ann      math     4.5
Mark     math     NaN
Max      rus      2.0
Mikhail  rus     10.0
Nina     rus      5.0
Pasha    eng      7.0
Sasha    eng      8.0
         math     7.0
Valera   eng      6.0
Name: Mark_1, dtype: float64

# Группировка по нескольким агрегирующим функциям

df.groupby('column_for_agg').agg({'column_for_create_cal_1' : function1, 
                                  'column_for_create_cal_2' : [function2, function3]})

где 
   * _column_for_agg_ - столбец, по которому будем агрегировать 
   * _column_for_create_cal_1_ - столбец, по которому будем вычислять значение
   * _column_for_create_cal_2_ - столбец, по которому будем вычислять значение
   * _function1_ - агрегирующая функция
   * _function2_ - агрегирующая функция
   * _function3_ - агрегирующая функция

In [12]:
df.groupby('Names').agg({'Mark_1' : 'mean', 
                         'Mark_2' : [np.mean, 'sum']})

Mark_1 Mark_2      
          mean   mean   sum
Names                      
Ann        4.5    7.0  14.0
Mark       NaN    NaN   0.0
Max        2.0    7.0   7.0
Mikhail   10.0    2.0   2.0
Nina       5.0    9.0   9.0
Pasha      7.0   10.0  10.0
Sasha      7.5    7.0  14.0
Valera     6.0    6.0   6.0

**!!!НЕ ЗАБЫВАЕМ!!!**

1) Если к одному столбцу хотим применить несколько агрегирующих функций, то нужно брать их в [] скобки

2) В данном услучае, из-за того, что мы применяем к одному полю разные агрегирующие функции, у нас появляется мультииндекс :) Чтобы этого избежать, можно переименовать поля через df.columns = list_column_names, где list_column_names - список из названий полей

In [13]:
df_groupby = df.groupby('Names').agg({'Mark_1' : 'mean', 
                         'Mark_2' : [np.mean, 'sum']})
df_groupby

Mark_1 Mark_2      
          mean   mean   sum
Names                      
Ann        4.5    7.0  14.0
Mark       NaN    NaN   0.0
Max        2.0    7.0   7.0
Mikhail   10.0    2.0   2.0
Nina       5.0    9.0   9.0
Pasha      7.0   10.0  10.0
Sasha      7.5    7.0  14.0
Valera     6.0    6.0   6.0

In [14]:
df_groupby.columns

MultiIndex([('Mark_1', 'mean'),
            ('Mark_2', 'mean'),
            ('Mark_2',  'sum')],
           )

In [15]:
df_groupby.columns = ['Mark_1_mean', 'Mark_2_mean', 'Mark_2_sum']

In [16]:
df_groupby

,Mark_1_mean,Mark_2_mean,Mark_2_sum
Names,,,
Ann,4.5,7.0,14.0
Mark,NaN,NaN,0.0
Max,2.0,7.0,7.0
Mikhail,10.0,2.0,2.0
Nina,5.0,9.0,9.0
Pasha,7.0,10.0,10.0
Sasha,7.5,7.0,14.0
Valera,6.0,6.0,6.0
